In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from PIL import Image
import random
from tqdm import tqdm 

IMG_SIZE = (48, 48)
TEST_SIZE = 0.2
RANDOM_STATE = 40
BATCH_SIZE = 64
EPOCHS = 50

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)


def load_images():
    images = []
    labels = []
    
    fire_dirs = [
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_dataset/fire_images",
        "C:/Users/hp/Python/Fire_Detection/Dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test/fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_data/train/fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_data/test/fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_data/val/fire"
    ]
    
    non_fire_dirs = [
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_dataset/non_fire_images",
        "C:/Users/hp/Python/Fire_Detection/Dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train/non_fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test/non_fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_data/train/non_fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_data/test/non_fire",
        "C:/Users/hp/Python/Fire_Detection/Dataset/fire_data/val/non_fire"
    ]

    def read_img(img_path):
        # OpenCV to read img
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            return img
            
        # PIL otherwise
        try:
            with Image.open(img_path) as pil_img:
                if pil_img.mode != 'L':
                    pil_img = pil_img.convert('L')
                return np.array(pil_img)
        except:
            return None

    # fire images
    for fire_dir in fire_dirs:
        if not os.path.exists(fire_dir):
            print(f"Warning: Directory not found - {fire_dir}")
            continue
            
        for filename in tqdm(os.listdir(fire_dir), desc=f"Loading {os.path.basename(fire_dir)}"):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
                
            img_path = os.path.join(fire_dir, filename)
            img = read_img(img_path)
            
            if img is not None:
                try:
                    img = cv2.resize(img, IMG_SIZE)
                    images.append(img)
                    labels.append(1) 
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
            else:
                print(f"Could not read: {filename}")

    # non-fire images
    for non_fire_dir in non_fire_dirs:
        if not os.path.exists(non_fire_dir):
            print(f"Warning: Directory not found - {non_fire_dir}")
            continue
            
        for filename in tqdm(os.listdir(non_fire_dir), desc=f"Loading {os.path.basename(non_fire_dir)}"):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
                
            img_path = os.path.join(non_fire_dir, filename)
            img = read_img(img_path)
            
            if img is not None:
                try:
                    img = cv2.resize(img, IMG_SIZE)
                    images.append(img)
                    labels.append(0) 
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
            else:
                print(f"Could not read: {filename}")

    return np.array(images), np.array(labels)

print("Loading images...")
images, labels = load_images() 

unique, counts = np.unique(labels, return_counts=True)
print(f"\nClass distribution: {dict(zip(['Non-Fire', 'Fire'], counts))}")
print(f"Total images loaded: {len(images)}")

X_train, X_test, Y_train, Y_test = train_test_split(
    images, labels, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=labels)

X_train = X_train.reshape(X_train.shape[0], IMG_SIZE[0], IMG_SIZE[1], 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], IMG_SIZE[0], IMG_SIZE[1], 1).astype('float32') / 255

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')


Loading images...


Loading non_fire: 100%|██████████| 267/267 [00:00<00:00, 711.82it/s]



Class distribution: {'Non-Fire': 17174, 'Fire': 17453}
Total images loaded: 34627


In [ ]:
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1), 
               kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(2, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy', 
                        tf.keras.metrics.Precision(name='precision_fire', class_id=1),
                        tf.keras.metrics.Recall(name='recall_fire', class_id=1)
])
    return model

model = create_model()
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
]

print("\nTraining model...")
history = model.fit(
    train_datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, Y_test),
    callbacks=callbacks)

print("\nEvaluating model...")
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, Y_test)
print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Test Precision: {test_precision:.4f} (Higher means fewer false positives)")
print(f"Test Recall: {test_recall:.4f} (Higher means fewer false negatives)")

model.save("improved_fire_detection_model.h5")
print("\nModel saved as 'improved_fire_detection_model.h5'")

class_indices = {'non_fire': 0, 'fire': 1}
import json
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)

C:\Users\hp\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 46, 46, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 21, 21, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 619,650 (2.36 MB)

 Trainable params: 618,690 (2.36 MB)

...

C:\Users\hp\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


432/432 ━━━━━━━━━━━━━━━━━━━━ 68s 145ms/step - accuracy: 0.7911 - loss: 1.0619 - precision_fire: 0.8003 - recall_fire: 0.7788 - val_accuracy: 0.7407 - val_loss: 0.9481 - val_precision_fire: 0.8424 - val_recall_fire: 0.5973 - learning_rate: 0.0010
Epoch 2/50
  1/432 ━━━━━━━━━━━━━━━━━━━━ 51s 120ms/step - accuracy: 0.9375 - loss: 0.5411 - precision_fire: 0.9667 - recall_fire: 0.9062

C:\Users\hp\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


432/432 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9375 - loss: 0.5411 - precision_fire: 0.9667 - recall_fire: 0.9062 - val_accuracy: 0.7444 - val_loss: 0.9378 - val_precision_fire: 0.8578 - val_recall_fire: 0.5909 - learning_rate: 0.0010
Epoch 3/50
432/432 ━━━━━━━━━━━━━━━━━━━━ 62s 142ms/step - accuracy: 0.8682 - loss: 0.6286 - precision_fire: 0.8733 - recall_fire: 0.8637 - val_accuracy: 0.7551 - val_loss: 0.8733 - val_precision_fire: 0.8571 - val_recall_fire: 0.6170 - learning_rate: 0.0010
Epoch 4/50
432/432 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8438 - loss: 0.5111 - precision_fire: 0.9118 - recall_fire: 0.8158 - val_accuracy: 0.7544 - val_loss: 0.8534 - val_precision_fire: 0.8385 - val_recall_fire: 0.6351 - learning_rate: 0.0010
Epoch 5/50
432/432 ━━━━━━━━━━━━━━━━━━━━ 61s 142ms/step - accuracy: 0.8785 - loss: 0.4850 - precision_fire: 0.8810 - recall_fire: 0.8775 - val_accuracy: 0.8979 - val_loss: 0.3957 - val_precision_fire: 0.8724 - val_recall_fire: 0.9341 - learning_ra


Test Accuracy: 0.8979
Test Precision: 0.8724 (Higher means fewer false positives)
Test Recall: 0.9341 (Higher means fewer false negatives)

Model saved as 'improved_fire_detection_model.h5'


In [18]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

Y_pred_probs = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred_probs, axis=1)
Y_true_classes = np.argmax(Y_test, axis=1)

print(confusion_matrix(Y_true_classes, Y_pred_classes))
print(classification_report(Y_true_classes, Y_pred_classes, target_names=['Non-Fire', 'Fire']))


217/217 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step
[[2958  477]
 [ 230 3261]]
              precision    recall  f1-score   support

    Non-Fire       0.93      0.86      0.89      3435
        Fire       0.87      0.93      0.90      3491

    accuracy                           0.90      6926
   macro avg       0.90      0.90      0.90      6926
weighted avg       0.90      0.90      0.90      6926

